# Epochs and Julian Dates

The representation of an epoch, that is of a specific point in time, be it in the future or in the past, can be rather confusing. In `pykep` we opted to offer the dedicated class {class}`pykep.epoch` that takes care to offer a simple interface and, under the hoods, interfaces seamlessly both to the c++ `std::chrono`  library and the python {class}`numpy.datetime` module. Find it in our API documentation under the section "[Relatated to epochs](../epoch.rst)"

Let us briefly show its interface.

An `epoch` may be created in one of four ways: 

1. constructing one from a Julian Date (i.e. a float representing the number of days passed from some historical date).
2. constructing one from a datetime object.
3. constructing directly from an ISO 8601 string.
4. requesting the current date from the {func}`pykep.utc_now` function.

:::{note}
**MJD2000** is the Default Julian Date. When not specified othewise by the user, in the context of epoch arithmetics a float will always be considered by `pykep` as a Modified Julian Date 2000, i.e. as the number of days from `2000-01-01T00:00:00.000000`, or as days if it represents a duration.

:::{note}
The date in `pykep` **does** account for leap seconds. If the user wishes to use the exact ISO 8601 representation of some epoch, also including leap seconds, he will have to account for the offset himself. As of of 2023 this may account to maximum 28 seconds. [More info](https://en.wikipedia.org/wiki/Leap_second) on leap seconds.

In [1]:
import pykep as pk
import datetime

## Julian dates

In [2]:
ep = pk.epoch(0.)

we can print this on screen:

In [3]:
print(ep)

2000-01-01T00:00:00.000000


.. or instantiate an epoch by explicitly mentioning the Julian Date type:

In [4]:
ep = pk.epoch(0., pk.epoch.julian_type.MJD2000)
print(ep)

2000-01-01T00:00:00.000000


.. or use a different Julian Date than the default MJD2000:

In [5]:
ep = pk.epoch(2460676.5000000, pk.epoch.julian_type.JD)
print(ep)

2025-01-01T00:00:00.000000


:::{note}
`pykep` supports the following Julian Dates MJD2000 (the default), MJD and JD. (see {class}`pykep.epoch.julian_type`)

We may also request an epoch corresponding to the current UTC time:

In [6]:
ep = pk.utc_now()
print(ep)

2023-10-16T13:24:28.532877


or construct it from an iso string:


In [7]:
ep = pk.epoch("2023-10-28T00:01:02.12")
print(ep)


2023-10-28T00:01:02.120000


## Datetime interoperability

If we have a datetime object from python builtin datetime library we can construct an epoch with it:

In [8]:
dt = datetime.datetime(year=2033, month=11, day=12, hour=12, minute=22, second=12, microsecond=14532)
ep = pk.epoch(dt)
print(ep)

2033-11-12T12:22:12.014532


and convert it, when needed, to a julian representation:

In [9]:
print(ep.mjd)

63913.51541683486


## The epoch math
Additions and subtractions are allowed between epochs and floats or `datetime.timedelta`. 
When using floats days are always assumed.

In [10]:
ep = pk.epoch(0)
ep = ep + 21.2353525 # This will be interpreted as days
print(ep)

2000-01-22T05:38:54.456000


In [11]:
ep = pk.epoch(0)
ep = ep + datetime.timedelta(hours = 5, seconds=54, days=21, minutes=38, microseconds=456000) # This will be interpreted as days
print(ep)

2000-01-22T05:38:54.456000


Some basic comparison operators are also allowed and may turn handy!

In [12]:
print(ep < ep + 4)
print(ep == ep + datetime.timedelta(days=32) - 32)

True
True


In [57]:
from sgp4.api import Satrec
from sgp4 import exporter
import pykep as pk
import numpy as np

class tle:
    """__init__(line1, line2)

    This User Defined Planet (UDPLA) represents a satellite orbiting the Earth and defined in the TLE format
    and propagated using the SGP4 propagator.

    .. note::
       The resulting ephemerides will be returned in SI units and in the True Equator Mean Equinox (TEME) reference frame

    """
    def __init__(self, line1, line2):
        import pykep as pk
        self.satellite = Satrec.twoline2rv(line1, line2)
        self.e = 0
        self.ref_epoch = pk.epoch(self.satellite.jdsatepoch + self.satellite.jdsatepochF, pk.epoch.julian_type.JD)
    def eph(self, ep):
        jd = ep.jd
        jd_i = int(jd)
        jd_fr = jd-jd_i
        self.e, r, v = self.satellite.sgp4(jd_i, jd_fr)
        return [[it*1000 for it in r], [it*1000 for it in v]]
    def eph_v(self, eps):
        jds = [ep.jd for ep in eps]
        jd_is = [int(item) for item in jds]
        jd_frs = [a-b for a,b in zip(jds,jd_is)]
        self.e, r, v = self.satellite.sgp4_array(np.array(jd_is), np.array(jd_frs))
        rv = np.hstack((r,v))
        return [rv[:,:3], rv[:,3:]]
    def get_name(self):
        return self.satellite.satnum_str + " - SGP4"
    def get_extra_info(self):
        line1, line2 = exporter.export_tle(self.satellite)
        return "TLE line1: " + line1 + "\nTLE line2: " + line2 
    def get_mu_central_body(self):
        return pk.MU_EARTH

In [58]:
line1 = '1 25544U 98067A   19343.69339541  .00001764  00000-0  38792-4 0  9991'
line2 = '2 25544  51.6439 211.2001 0007417  17.6667  85.6398 15.50103472202482'
udpla = tle(line1, line2)
eps = [pk.epoch(when%1000.2) for when in range(100000)]
satellite = Satrec.twoline2rv(line1, line2)


In [59]:
pla = pk.planet(udpla)

In [60]:
%%time
for ep in eps:
    udpla.eph(ep)

CPU times: user 111 ms, sys: 1.85 ms, total: 112 ms
Wall time: 111 ms


In [61]:
%%time
for ep in eps:
    pla.eph(ep)

CPU times: user 271 ms, sys: 1.72 ms, total: 273 ms
Wall time: 272 ms


In [62]:
satellite = Satrec.twoline2rv(line1, line2)

In [63]:
%%timeit
pla.eph_v(eps)

RuntimeError: Unable to cast Python instance of type <class 'list'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)

In [38]:
%%timeit
for ep in eps:
    jd = ep.jd
    jd_i = int(jd)
    jd_fr = jd-jd_i
    e, r, v = satellite.sgp4(jd_i, jd_fr)
    [it*1000 for it in r], [it*1000 for it in v]

96.8 ms ± 465 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
%%timeit
satellite.sgp4(27000, 0.0)

479 ns ± 2.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [40]:
jds = [item.jd for item in eps]
jd_is = [int(item) for item in jds]
jd_frs = [a-b for a,b in zip(jds, jd_is)]
e, r, v = satellite.sgp4_array(np.array(jd_is), np.array(jd_frs))


In [41]:
r

array([[-1342.62309448,  5032.4243746 , -4439.55125039],
       [-3719.45084139, -2366.62425989,  5224.2841185 ],
       [ 5813.38678479, -2339.49077054, -2721.45373998],
       ...,
       [ -216.4187955 , -4542.78037437,  5108.56384263],
       [ 5547.54702119,  3541.24918466, -1846.88529286],
       [-6180.37780609,   698.81920703, -2867.8951121 ]])

In [43]:
type(r[0])

numpy.ndarray

In [64]:
np.hstack((r,v))

array([[-1.34262309e+03,  5.03242437e+03, -4.43955125e+03,
        -6.74956322e+00,  1.16902739e+00,  3.35423473e+00],
       [-3.71945084e+03, -2.36662426e+03,  5.22428412e+03,
         5.28445236e+00, -5.34777953e+00,  1.32628911e+00],
       [ 5.81338678e+03, -2.33949077e+03, -2.72145374e+03,
        -1.46666466e-01,  5.63187335e+00, -5.16532254e+00],
       ...,
       [-2.16418796e+02, -4.54278037e+03,  5.10856384e+03,
         7.21504628e+00,  1.70875772e+00,  1.81082782e+00],
       [ 5.54754702e+03,  3.54124918e+03, -1.84688529e+03,
        -3.97684598e+00,  3.30300205e+00, -5.62835309e+00],
       [-6.18037781e+03,  6.98819207e+02, -2.86789511e+03,
        -2.90631033e+00, -4.91484466e+00,  5.05528530e+00]])

In [47]:
r[0]

array([-1342.62309448,  5032.4243746 , -4439.55125039])

In [48]:
v[0]

array([-6.74956322,  1.16902739,  3.35423473])